In [2]:
#Apache Spark : Genel amaçlı paralel veri işleme frameworku
#Hadoop üzerinde çalışabilir
#Büyük Veri İşleme, Büyük Veri ile Makine Öğrenmesi ===> HADOOP + Spark
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [1,963 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13

In [3]:
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

In [4]:
dolar = sc.textFile("DolarVerileri.txt")
dolar.take(5)

def splitTake(line):
    arr = line.split('\t')
    order=int(arr[0])
    date=arr[1]
    currency=arr[2].replace(".","").replace(",",".")
    return order,(date,currency)

def addsplitTake(line):
    arr = line.split('\t')
    order=int(arr[0]) + 1
    date=arr[1] 
    currency=arr[2].replace(".","").replace(",",".")
    return order,(date,currency)

dolarRDD1 = dolar.map(lambda line : splitTake(line)).filter(lambda x: x[1][1]!="")

dolarRDD2 = dolar.map(lambda line : addsplitTake(line)).filter(lambda x: x[1][1]!="")

#print(dolarRDD1.take(2))
#print(dolarRDD2.take(2))

joinedRDD = dolarRDD2.join(dolarRDD1)
#joinedRDD = sc.parallelize(joinedRDD)
#joinedRDD.take(5)


def ratiocalculator(data):
 dolar1 = data[1][0][1]
 date1 = data[1][0][0]
 dolar2 = data[1][1][1]
 date2 = data[1][1][0]
 diff = (float(dolar2)-float(dolar1))/float(dolar1)
 return date1,dolar1,dolar2,date2,diff
res = joinedRDD.map(lambda line : ratiocalculator(line)).sortBy(lambda x: x[4],ascending=False)
res.take(5)

[('19-08-1960', '2.80', '9.00', '22-08-1960', 2.2142857142857144),
 ('24-01-1980', '35.00', '70.00', '25-01-1980', 1.0),
 ('07-08-1970', '9.00', '14.85', '10-08-1970', 0.6499999999999999),
 ('22-02-2001', '685391.00', '957879.00', '23-02-2001', 0.39756576902818974),
 ('05-04-1994', '23031.84', '31988.89', '06-04-1994', 0.38889858561018137)]